In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import as_strided
import psycopg2
import copy
import os
import sys
import simplejson as json
from typing import Dict
pd.options.plotting.backend = "plotly"
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import math
import re
from decimal import Decimal

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
print(sys.path)

from ddx.auditor.auditor_driver import AuditorDriver
from ddx.auditor.state.identifiers.strategy_identifier import StrategyIdentifier
from ddx.auditor.state.identifiers.position_identifier import PositionIdentifier
from ddx.auditor.state.identifiers.organic_insurance_fund_identifier import OrganicInsuranceFundIdentifier


/var/folders/wm/8jvz1t910hjfsxvjbqgnhsm00000gn/T/ipykernel_1816/2366266365.py:12: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/wm/8jvz1t910hjfsxvjbqgnhsm00000gn/T/ipykernel_1816/2366266365.py:13: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


['/Users/adityapalepu/projects/derivadex/rust-packages/ddx-python/python/notebooks', '/Users/adityapalepu/projects/derivadex/rust-packages/ddx-python/notebooks', '/Users/adityapalepu/projects/derivadex/rust-packages/ddx-python', '/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python38.zip', '/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8', '/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/lib-dynload', '', '/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/site-packages', '/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/site-packages/IPython/extensions', '/Users/adityapalepu/.ipython']


# Multi node diverge

## Load data

In [5]:
db_names = ['node0', 'node1', 'node2']
db_host = 'localhost'
db_password = os.environ['PG_JUPYTER_READONLY_PW']
db_port = os.environ['PGPORT']
db_username = os.environ['PG_JUPYTER_READONLY_ROLE']

In [29]:
# Get liquidation table
get_tx_log_table_sql = """SELECT * FROM tx_log"""
tx_log_df_dict = {}

for db_name in db_names:
    # Connect to an existing database
    try:
        connection = psycopg2.connect(user=db_username,
                                      password=db_password,
                                      host=db_host,
                                      port=db_port,
                                      database=db_name)

        # Create a cursor to perform database operations
        cursor = connection.cursor()
        # Print PostgreSQL details
        print("PostgreSQL server information")
        print(connection.get_dsn_parameters(), "\n")
        # Executing a SQL query
        cursor.execute("SELECT version();")
        # Fetch result
        record = cursor.fetchone()
        print("You are connected to - ", record, "\n")
    finally:
        if connection:
            # cursor.close()
            # connection.close()
            print("PostgreSQL connection is closed (but not really)")
            
    tx_log_df_dict[db_name] = pd.read_sql_query(get_tx_log_table_sql, connection)
    tx_log_df_dict[db_name]['state_root_hash'] = tx_log_df_dict[db_name]['state_root_hash'].apply(lambda x: f'0x{x.hex()}')
    tx_log_df_dict[db_name].sort_values(['request_index'], ascending=[1], inplace=True)

PostgreSQL server information
{'user': 'postgres', 'dbname': 'node0', 'host': 'localhost', 'port': '5434', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.7 on x86_64-pc-linux-musl, compiled by gcc (Alpine 11.2.1_git20220219) 11.2.1 20220219, 64-bit',) 

PostgreSQL connection is closed (but not really)
PostgreSQL server information
{'user': 'postgres', 'dbname': 'node1', 'host': 'localhost', 'port': '5434', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.7 on x86_64-pc-linux-musl, compiled by gcc (Alpine 11.2.1_git20220219) 11.2.1 20220219, 64-bit',) 

PostgreSQL connection is closed (but not really)
PostgreSQL server information
{'user': 'postgres', 'dbname': 'node2', 'host': 'localhost', 'port': '54

In [30]:
merged_df = pd.merge(
    tx_log_df_dict['node0'],
    tx_log_df_dict['node1'][['request_index', 'state_root_hash', 'event']],
    left_on=['request_index'],
    right_on=['request_index'],
    suffixes=['_node0', '_node1'],
    how='outer',
    
).sort_values(['request_index'], ascending=[1])
merged_df.head()

,epoch_id,tx_ordinal,request_index,batch_id,time_value,time_stamp,state_root_hash_node0,event_kind,event_node0,state_root_hash_node1,event_node1
0,0,0,6,6,1,1655618403,0x00000000000000000000000000000000000000000000...,100,"{'c': {'kind': 'Genesis', 'stateRootHash': '0x...",0x00000000000000000000000000000000000000000000...,"{'c': {'kind': 'Genesis', 'stateRootHash': '0x..."
1,1,0,7,7,1,1655618403,0x6688a02e99a5df8246b8dad32a49d5edbe5649c08559...,60,"{'c': {'url': 'http://operator-node0:8080', 'a...",0x6688a02e99a5df8246b8dad32a49d5edbe5649c08559...,"{'c': {'url': 'http://operator-node0:8080', 'a..."
2,1,1,56,56,31,1655618433,0xc4bc8a4743a8d8b20ebe06432e1acf5d7e51fc1dfcae...,9,"{'c': {'BTCP': {'ema': '0', 'ordinal': 1, '...",0xc4bc8a4743a8d8b20ebe06432e1acf5d7e51fc1dfcae...,"{'c': {'BTCP': {'ema': '0', 'ordinal': 1, '..."
3,1,2,92,92,40,1655618442,0x043d38fd029c5c41d32b87b680e31d3c06c173b79d4c...,5,"{'c': {'amount': '1000', 'trader': '0x001c6191...",0x043d38fd029c5c41d32b87b680e31d3c06c173b79d4c...,"{'c': {'amount': '1000', 'trader': '0x001c6191..."
4,1,3,96,96,40,1655618442,0xd0a16b3588fba79c832e737ea8a258ea4299a44ccaef...,5,"{'c': {'amount': '1000', 'trader': '0x00123553...",0xd0a16b3588fba79c832e737ea8a258ea4299a44ccaef...,"{'c': {'amount': '1000', 'trader': '0x00123553..."


In [31]:
mismatching_txs_df = merged_df.query('state_root_hash_node0 != state_root_hash_node1')
mismatching_txs_df.head()

,epoch_id,tx_ordinal,request_index,batch_id,time_value,time_stamp,state_root_hash_node0,event_kind,event_node0,state_root_hash_node1,event_node1
2273,2,7,8379,8379,1205,1655619607,0x653e95d40213770592e850971b4fe139012d7fbccc73...,2,"{'c': [{'side': 'Ask', 'price': '4151', 'amoun...",0x5faa42e8a6dca4a45f69278bd1564a1368ec7b15a266...,"{'c': [{'side': 'Ask', 'price': '4151', 'amoun..."
2274,2,8,8380,8380,1205,1655619607,0xd7e9876702642a436076622794363f82c864de404af2...,2,"{'c': [{'side': 'Bid', 'price': '4113.8', 'amo...",0xd7045d6901e84060a6c4cfe5306439750ce38ef69de1...,"{'c': [{'side': 'Bid', 'price': '4113.8', 'amo..."
2275,2,9,8381,8381,1205,1655619607,0xc66e4ad649acccdded5059edd79a34e267c1fadcaf47...,2,"{'c': [{'side': 'Ask', 'price': '4171.7', 'amo...",0xf444e9e08ab1b6d41febdcb269f5bced437cf4c55717...,"{'c': [{'side': 'Ask', 'price': '4171.7', 'amo..."
2276,2,10,8382,8382,1205,1655619607,0x1a623e241dbed27d9ff852ef82655ab999cf43940c21...,3,"{'c': {'amount': '5.4', 'symbol': 'BTCP', '...",0xd77d98347cc5898ca10e3c1b4528edd200a7b12f5c06...,"{'c': {'amount': '5.4', 'symbol': 'BTCP', '..."
2277,2,11,8385,8385,1205,1655619607,0x6279c1d8dbabc5ea1ec8c7477b2c63968062a65f19de...,5,"{'c': {'amount': '1000', 'trader': '0x0060e7f5...",0x9e03f3edc3fb1855892af632278364d2a52e3880ea4c...,"{'c': {'amount': '1000', 'trader': '0x0060e7f5..."


In [32]:
first_mismatching_request = mismatching_txs_df.iloc[0]['request_index']
first_mismatching_request

8379

In [33]:
s = merged_df[merged_df['request_index'] < first_mismatching_request].iloc[-1]
mishandled_tx

epoch_id                                                                 2
tx_ordinal                                                               6
request_index                                                         8378
batch_id                                                              8378
time_value                                                            1205
time_stamp                                                      1655619607
state_root_hash_node0    0x2cb9f979a731b7066ad2bba3fb3a3f18bf0660ff6e48...
event_kind                                                               2
event_node0              {'c': [{'side': 'Bid', 'price': '4134.5', 'amo...
state_root_hash_node1    0x2cb9f979a731b7066ad2bba3fb3a3f18bf0660ff6e48...
event_node1              {'c': [{'side': 'Bid', 'price': '4134.5', 'amo...
Name: 2272, dtype: object

In [34]:
mismatching_tx_node0 = merged_df[merged_df['request_index'] < first_mismatching_request].iloc[-1]['event_node0']
mismatching_tx_node0

{'c': [{'side': 'Bid',
   'price': '4134.5',
   'amount': '25.6',
   'symbol': 'ETHP',
   'orderHash': '0xcb81448799375c04e10f328d1a12621ae1b04476a29331c28d',
   'strategyId': 'main',
   'bookOrdinal': 264,
   'traderAddress': '0x00accf7803c88b05846c46946d7b46418a9adad578'},
  {'ETHP': {'ema': '-117.005333858916652201',
    'ordinal': 359,
    'indexPrice': '4092.4',
    'indexPriceHash': '0x7b45fdf18e52e5518b66b753710483060a03a358f62f0a5236'}},
  []],
 't': 'Post'}

In [35]:
mismatching_tx_node1 = merged_df[merged_df['request_index'] < first_mismatching_request].iloc[-1]['event_node1']
mismatching_tx_node1

{'c': [{'side': 'Bid',
   'price': '4134.5',
   'amount': '25.6',
   'symbol': 'ETHP',
   'orderHash': '0xcb81448799375c04e10f328d1a12621ae1b04476a29331c28d',
   'strategyId': 'main',
   'bookOrdinal': 263,
   'traderAddress': '0x00accf7803c88b05846c46946d7b46418a9adad578'},
  {'ETHP': {'ema': '-117.005333858916652201',
    'ordinal': 359,
    'indexPrice': '4092.4',
    'indexPriceHash': '0x7b45fdf18e52e5518b66b753710483060a03a358f62f0a5236'}},
  []],
 't': 'Post'}

In [40]:
mask = (tx_log_df_dict['node0']['event_kind'] == 2) & (tx_log_df_dict['node0']['request_index'] < first_mismatching_request)
tx_log_df_dict['node0'][mask]

[autoreload of ddx.auditor.transactions.event_types failed: Traceback (most recent call last):
  File "/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/Users/adityapalepu/opt/anaconda3/envs/derivadex_b/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __

,epoch_id,tx_ordinal,request_index,batch_id,time_value,time_stamp,state_root_hash,event_kind,event
41,1,29,261,261,65,1655618467,0x9df8a50cd3d7fe8863d64eff7b38c6b379944745d3ee...,2,"{'c': [{'side': 'Bid', 'price': '4361', 'amoun..."
69,1,30,262,262,65,1655618467,0x625e11f6dc17aa25c9bc45d2453de7466506cbb0a0ef...,2,"{'c': [{'side': 'Bid', 'price': '4356.7', 'amo..."
51,1,31,263,263,65,1655618467,0x0cc698d376249427769b6fe678a72812eb0c51e8d899...,2,"{'c': [{'side': 'Ask', 'price': '4378.5', 'amo..."
20,1,32,265,265,66,1655618468,0xab40b3862ba0e0fd4ae980b0ece71dfb35cf84362a25...,2,"{'c': [{'side': 'Ask', 'price': '4387.2', 'amo..."
70,1,33,266,266,66,1655618468,0x24c9f907900a564e561467fe23b40ef0bf844066d8d2...,2,"{'c': [{'side': 'Bid', 'price': '4330.5', 'amo..."
...,...,...,...,...,...,...,...,...,...
2294,1,2227,8183,8183,1175,1655619577,0x5c200ac2b42e451e061518d68a1f9e2362c0b5101fca...,2,"{'c': [{'side': 'Bid', 'price': '3615.2', 'amo..."
2283,1,2228,8184,8184,1175,1655619577,0x3c6bf6c3e05f24672e6c0f225ca90bc76034ddd8ab33...,2,"{'c': [{'side': 'Bid', 'price': '32016', 'amou..."
2257,1,2229,8185,8185,1175,1655619577,0xda0b376f7d749b68a4ce29ddd8d5ee212972ff6df367...,2,"{'c': [{'side': 'Ask', 'price': '32144', 'amou..."
2260,1,2235,8208,8208,1180,1655619582,0xd6b9040db6e349a00d475b163dd9eb45a159dfb463d4...,2,"{'c': [{'side': 'Ask', 'price': '72521', 'amou..."


In [61]:
mask = (tx_log_df_dict['node1']['event_kind'] == 2) & (tx_log_df_dict['node1']['request_index'] < first_mismatching_request)
tx_log_df_dict['node1'][mask].iloc[-5]['event']

{'c': [{'side': 'Bid',
   'price': '3615.2',
   'amount': '51.5',
   'symbol': 'ETHP',
   'orderHash': '0x673eb555d884e6320e740554485125e9877a6481928f985e6f',
   'strategyId': 'main',
   'bookOrdinal': 263,
   'traderAddress': '0x00accf7803c88b05846c46946d7b46418a9adad578'},
  {},
  []],
 't': 'Post'}

In [64]:
tx_log_df_dict['node1'][mask].iloc[-1]['event']

{'c': [{'side': 'Bid',
   'price': '4134.5',
   'amount': '25.6',
   'symbol': 'ETHP',
   'orderHash': '0xcb81448799375c04e10f328d1a12621ae1b04476a29331c28d',
   'strategyId': 'main',
   'bookOrdinal': 263,
   'traderAddress': '0x00accf7803c88b05846c46946d7b46418a9adad578'},
  {'ETHP': {'ema': '-117.005333858916652201',
    'ordinal': 359,
    'indexPrice': '4092.4',
    'indexPriceHash': '0x7b45fdf18e52e5518b66b753710483060a03a358f62f0a5236'}},
  []],
 't': 'Post'}